<a href="https://colab.research.google.com/github/yuzukiiii16/ml-course/blob/main/%E3%80%90Ch2_b%E3%80%91Linear_Regression%EF%BC%BF%E7%AC%ACO%E7%B5%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>說明：</h1>
<h3>你需要修改的地方會用底下的標記框出來，請依照註解的提示修改</h3>



```
# ==========================


# ==========================
```



# 匯入必要套件

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

#匯入初步處理好的資料


In [ ]:
FILE_NAME="New_York_Airbnb_Open_Data_2024_cleaned.csv"

!curl -L -o $FILE_NAME "https://drive.google.com/uc?export=download&id=1tMS63gsms0AEglD0Ac6WOrWEtaTvFsCX"


###資料下載後，檢查是否成功讀取，如果能看到資料代表成功

In [ ]:
df = pd.read_csv("New_York_Airbnb_Open_Data_2024_cleaned.csv", encoding="utf-8")
df.head()

# 利用相關係數觀察price與各變數之間的關係

說明：

* 大於0.7為高度相關，可以視為值得選用的特徵。

* 在0.3到0.7之間為中度相關，需視情況斟酌。

* 小於0.3則相關性低，通常可忽略該特徵。

* 線性回歸不只看相關性，有時加入更多特徵會改善模型表現，但過多特徵也可能導致效果變差。

In [ ]:
# 計算相關係數矩陣
correlation_matrix = df.corr()

# 顯示所有變數與 'price' 的相關係數
print("各變數與 price 的相關係數:")
print(correlation_matrix['price'].sort_values(ascending=False))

# 特徵工程
* 取 log  
* One-hot 編碼

### 取log

雖然已處理極端值，但資料可能仍有偏態。

In [ ]:
# 檢查price為常態還是偏態
plt.figure(figsize=(6,4))
sns.histplot(df['price'], bins=50, kde=True)
plt.show()

In [ ]:
# 說明: 請將 price 取對數（log），讓分布更平均。
df['log_price'] = np.log(df['price'])

### One-hot 編碼

類別型資料不具備大小順序，無法直接進行數學運算，為了讓模型正確處理，請使用 One-Hot Encoding

In [ ]:
# ==========================
# 請修改以下程式碼
# 說明:請找出類別欄位並進行One-Hot Encoding
#     如果有超過一個欄位，用逗號 , 連接，例如 ['col1', 'col2']

df = pd.get_dummies(df, columns=['']) # <- 將空字串替換成欄位名稱

# ==========================

In [ ]:
# 確認是否成功進行one-hot編碼
df.head(100)

# 進行線性回歸
# price為目標變項，自行選擇自變項（x）進行線性回歸

* 資料切分常見比例：訓練集 60–80%，測試集 20–40%
* 線性回歸不只看相關性，有時加入更多特徵會改善模型表現，但過多特徵也可能導致效果變差。

In [ ]:
# 類別欄位
new_columns = [col for col in df.columns if 'neighbourhood_group_' in col or 'room_type_' in col or 'neighbourhood_' in col]


# ==========================
# 請修改以下程式碼
# 說明: 請根據你觀察的結果，選擇你適合的「特徵」，填入下面的 [ ] 中：
# 不需要寫類別欄位，類別欄位已寫在new_colums

features_lr = [ ] + new_columns

# ==========================

X_lr = df[features_lr]
y_lr = df['log_price']

# 分割訓練與測試資料（80% 訓練，20% 測試），可自行修改 test_size
X_train, X_test, y_train, y_test = train_test_split(X_lr, y_lr, test_size=0.2, random_state=42)

# 建立並訓練模型
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

顯示斜率和截距

In [ ]:
print("斜率:")
for feat, coef in zip(features_lr, lr_model.coef_):
    print(f"{feat}: {coef:.4f}")
print(f"截距: {lr_model.intercept_:.4f}")

計算訓練和測試資料的 R², MSE 和 RMSE

In [ ]:
# 預測訓練資料
y_train_pred = lr_model.predict(X_train)
# 預測測試資料
y_test_pred = lr_model.predict(X_test)

# 還原為實際價格
y_train_pred_price = np.exp(y_train_pred)
y_test_pred_price = np.exp(y_test_pred)
y_train_true_price = np.exp(y_train)
y_test_true_price = np.exp(y_test)

# 評估模型（訓練資料）
r2_train = lr_model.score(X_train, y_train)
mse_train_price = mean_squared_error(y_train_true_price, y_train_pred_price)
rmse_train_price = np.sqrt(mse_train_price)

# 評估模型（測試資料）
r2_test = lr_model.score(X_test, y_test)
mse_test_price = mean_squared_error(y_test_true_price, y_test_pred_price)
rmse_test_price = np.sqrt(mse_test_price)

# 顯示結果
print(f"訓練資料的 R²: {r2_train:.4f}")
print(f"訓練資料的 MSE: {mse_train_price:.2f}")
print(f"訓練資料的 RMSE: {rmse_train_price:.2f}")
print(f"測試資料的 R²: {r2_test:.4f}")
print(f"測試資料的 MSE: {mse_test_price:.2f}")
print(f"測試資料的 RMSE: {rmse_test_price:.2f}")

### 線性回歸模型的預測效果：比較「實際房價」和「模型預測出來的房價」的接近程度。


In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.xlabel('Actual price')
plt.ylabel('Predicted price')
plt.title('Actual vs Predicted price')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()